# Init

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from datetime import date
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from bokeh.core.enums import SizingMode
from bokeh.layouts import layout, column, row, gridplot
from bokeh.models import ColumnDataSource, HoverTool, Slider, CustomJS, TextInput, Div, RangeSlider, DateRangeSlider, Select, Styles, Range1d, InlineStyleSheet, FuncTickFormatter, CustomJSTickFormatter, BasicTicker
from bokeh.plotting import figure, output_file, save, show

# Helpers

In [ ]:
# Index by year and quarter.
# Helper function for a couple
# of different datasets.
def monthToQuarter(qrt):
    month = ''
    if (qrt == 'Qtr1'):
        month = 'Jan'
    elif (qrt == 'Qtr2'):
        month = 'Apr'
    elif (qrt == 'Qtr3'):
        month = 'Aug'
    elif (qrt == 'Qtr4'):
        month = 'Oct'
    else:
        raise Exception('Cannot convert quarter to month')
    return month

# Adds a percent sign to a number.
def percent_formatter(x):
    return f"{x}%"

## Datasource

In [ ]:
# Use BLS-issued series ID.
CPI_SRC = 'data/CUUR0000AA0.xlsx'
UNEMP_SRC = 'data/LNS14000000.xlsx'
ECI_SRC = 'data/CIU2020000000000A.xlsx'
EMP_SRC = 'data/CES0000000001.xlsx'
PROD_SRC = 'data/PRS85006092.xlsx'

## Settings

In [ ]:
LONG_HISTORY_START_YEAR = 1954  # CPI and unemployment data go back to 1954. Base CPI = 1967
SHORT_HISTORY_START_YEAR = 2001. # Other data only go back to 2001.
END_YEAR = 2024
MEDIUM_TERM = '2001-01-01'  # Dotcom bust. Many charts begin here; leave as-is.
SHORT_TERM = '2008-01-01'  # Great recession.
SHORTER_TERM = '2020-01-01'  # Pandemic.

In [ ]:
DISABLE_PLOT = True

# Datasets

## CPI

### Dataset

In [ ]:
# Load datasource.
df_cpi_orig = pd.read_excel(CPI_SRC, skiprows=list(range(11)))
df_cpi_orig.drop(['HALF1', 'HALF2'], inplace=True, axis=1)

# CPI with 1967 dollars
df_cpi_orig = df_cpi_orig[df_cpi_orig['Year'] >= LONG_HISTORY_START_YEAR]

# Index by date.
df_cpi_melted = df_cpi_orig.melt(id_vars='Year', var_name='Month', value_name='Value')
df_cpi_melted['Date'] = df_cpi_melted['Year'].astype(str) + '-' + df_cpi_melted['Month']
df_cpi_melted['Date'] = pd.to_datetime(df_cpi_melted['Date'], format='%Y-%b')
df_cpi = df_cpi_melted.set_index('Date')
df_cpi = df_cpi.sort_index()

### Matplotlib

In [ ]:
if (DISABLE_PLOT == False):
    plt.yscale('log')
    plt.plot(df_cpi.index, df_cpi['Value'])

In [ ]:
if (DISABLE_PLOT == False):
    plt.yscale('log')
    plt.plot(df_cpi[df_cpi.index >= MEDIUM_TERM].index, df_cpi[df_cpi.index >= MEDIUM_TERM]['Value']) # Other charts start at 2001.

### Bokeh (CPI Level)

In [ ]:
# Plot setup.
cpi_plot = figure(
    height=400,
    width=800,
    title="CPI",
    x_axis_label='Date',
    y_axis_label='1967 dollars (%)',
    x_axis_type="datetime",
    y_axis_type="log",
    tools="",
)

# Date range widget.
cpi_date_range_slider = DateRangeSlider(
    title="Period",
    value=(date(LONG_HISTORY_START_YEAR, 1, 1), date(END_YEAR, 1, 1)), # Hard code for now.
    start=date(LONG_HISTORY_START_YEAR, 1, 1),
    end=date(END_YEAR, 1, 1),
    step=365,
)

# Bind slider to plot for auto updating.
cpi_date_range_slider.js_link('value', cpi_plot.x_range, 'start', attr_selector=0)
cpi_date_range_slider.js_link('value', cpi_plot.x_range, 'end', attr_selector=1)

# Data source.
cpi_source = ColumnDataSource(data={'x': df_cpi.index, 'y': df_cpi['Value'], 'smoothed_y': df_cpi['Value']})

# Line plot.
cpi_line = cpi_plot.line('x', 'y', source=cpi_source, line_width=1, line_alpha=0.6, legend_label='Actual')

# Smoothed line plot.
cpi_smoothed_line = cpi_plot.line('x', 'smoothed_y', source=cpi_source, line_width=2, color='red', legend_label='Smooth')

# Resize viewport widget.
cpi_view_size_select = Select(title="Sizing mode", value="fixed", options=list(SizingMode), width=300)
cpi_view_size_select.js_link('value', cpi_plot, 'sizing_mode')

# Resize window widget.
cpi_window_callback = CustomJS(args=dict(source=cpi_source, smoothed_line=cpi_smoothed_line), code="""
    var data = source.data;
    var y = data['y'];
    var smoothed_y = data['smoothed_y'];
    var window_size = cb_obj.value;
    var N = y.length;
    
    // Simple moving average for smoothing
    for (var i = 0; i < N; i++) {
        var sum = 0.0;
        var count = 0;
        for (var j = Math.max(0, i - Math.floor(window_size / 2)); j < Math.min(N, i + Math.ceil(window_size / 2)); j++) {
            sum += y[j];
            count += 1;
        }
        smoothed_y[i] = sum / count;
    }
    
    // Update the plot
    smoothed_line.glyph.change.emit();
""")
cpi_window_size_slider = Slider(start=3, end=100, value=5, step=2, title="Average")
cpi_window_size_slider.js_on_change('value', cpi_window_callback)

# Line thickness widget.
cpi_thickness_slider_line = Slider(title="slide input", value=1.0, start=0.1, end=5.0, step=0.1)
cpi_thickness_slider_line_callback = CustomJS(args=dict(line=cpi_line, slider=cpi_thickness_slider_line), code="""
    line.glyph.line_width = slider.value;
""")
cpi_thickness_slider_line.js_on_change('value', cpi_thickness_slider_line_callback)

# Smoothed line thickness widget.
cpi_thickness_slider_smoothed = Slider(title="slide input", value=1.0, start=0.1, end=5.0, step=0.1)
cpi_thickness_slider_smoothed_callback = CustomJS(args=dict(line=cpi_smoothed_line, slider=cpi_thickness_slider_smoothed), code="""
    line.glyph.line_width = slider.value;
""")
cpi_thickness_slider_smoothed.js_on_change('value', cpi_thickness_slider_smoothed_callback)

# Build layout.
cpi_layout = column(
    cpi_window_size_slider,
    cpi_date_range_slider,
    cpi_view_size_select,
    cpi_thickness_slider_line,
    cpi_thickness_slider_smoothed,
    cpi_plot
)

### Inflation Rate Dataset

In [ ]:
# Copy df_cpi, add rate column.
df_cpi_rate = df_cpi.copy()
df_cpi_rate['Rate'] = ((df_cpi_rate['Value'] - df_cpi_rate['Value'].shift(1)) / df_cpi_rate['Value'].shift(1)) * 100
df_cpi_rate.loc['1954-01-01', 'Rate'] = df_cpi_rate.loc['1954-02-01', 'Rate']  # Backfill first month

### Bokeh (Inflation Rate)

In [ ]:
# Plot setup.
cpi_rate_plot = figure(
    height=400,
    width=800,
    title="CPI Rate",
    x_axis_label='Date',
    y_axis_label='Rate',
    x_axis_type="datetime",
    y_axis_type="linear",
    tools="",
)

# Date range widget.
cpi_rate_date_range_slider = DateRangeSlider(
    title="Period",
    value=(date(LONG_HISTORY_START_YEAR, 1, 1), date(END_YEAR, 1, 1)), # Hard code for now.
    start=date(LONG_HISTORY_START_YEAR, 1, 1),
    end=date(END_YEAR, 1, 1),
    step=365,
)

# Bind slider to plot for auto updating.
cpi_rate_date_range_slider.js_link('value', cpi_rate_plot.x_range, 'start', attr_selector=0)
cpi_rate_date_range_slider.js_link('value', cpi_rate_plot.x_range, 'end', attr_selector=1)

# Data source.
cpi_rate_source = ColumnDataSource(data={'x': df_cpi_rate.index, 'y': df_cpi_rate['Rate'], 'smoothed_y': df_cpi_rate['Rate']})

# Line plot.
cpi_rate_line = cpi_rate_plot.line('x', 'y', source=cpi_rate_source, line_width=1, line_alpha=0.2, legend_label='Actual')

# Smoothed line plot.
cpi_rate_smoothed_line = cpi_rate_plot.line('x', 'smoothed_y', source=cpi_rate_source, line_width=1.5, color='red', legend_label='Smooth')

# Resize viewport widget.
cpi_rate_view_size_select = Select(title="Sizing mode", value="fixed", options=list(SizingMode), width=300)
cpi_rate_view_size_select.js_link('value', cpi_rate_plot, 'sizing_mode')

# Resize window widget.
cpi_rate_window_callback = CustomJS(args=dict(source=cpi_rate_source, smoothed_line=cpi_rate_smoothed_line), code="""
    var data = source.data;
    var y = data['y'];
    var smoothed_y = data['smoothed_y'];
    var window_size = cb_obj.value;
    var N = y.length;
    
    // Simple moving average for smoothing
    for (var i = 0; i < N; i++) {
        var sum = 0.0;
        var count = 0;
        for (var j = Math.max(0, i - Math.floor(window_size / 2)); j < Math.min(N, i + Math.ceil(window_size / 2)); j++) {
            sum += y[j];
            count += 1;
        }
        smoothed_y[i] = sum / count;
    }
    
    // Update the plot
    smoothed_line.glyph.change.emit();
""")
cpi_rate_window_size_slider = Slider(start=1, end=100, value=1, step=1, title="Average")
cpi_rate_window_size_slider.js_on_change('value', cpi_rate_window_callback)

# Line thickness widget.
cpi_rate_thickness_slider_line = Slider(title="slide input", value=1.0, start=0.1, end=5.0, step=0.1)
cpi_rate_thickness_slider_line_callback = CustomJS(args=dict(line=cpi_rate_line, slider=cpi_rate_thickness_slider_line), code="""
    line.glyph.line_width = slider.value;
""")
cpi_rate_thickness_slider_line.js_on_change('value', cpi_rate_thickness_slider_line_callback)

# Smoothed line thickness widget.
cpi_rate_thickness_slider_smoothed = Slider(title="slide input", value=1.0, start=0.1, end=5.0, step=0.1)
cpi_rate_thickness_slider_smoothed_callback = CustomJS(args=dict(line=cpi_rate_smoothed_line, slider=cpi_rate_thickness_slider_smoothed), code="""
    line.glyph.line_width = slider.value;
""")
cpi_rate_thickness_slider_smoothed.js_on_change('value', cpi_rate_thickness_slider_smoothed_callback)

# Build layout.
cpi_rate_layout = column(
    cpi_rate_window_size_slider,
    cpi_rate_date_range_slider,
    cpi_rate_view_size_select,
    cpi_rate_thickness_slider_line,
    cpi_rate_thickness_slider_smoothed,
    cpi_rate_plot
)

## Unemployment

### Dataset

In [ ]:
# Load datasource.
df_unemp_orig = pd.read_excel(UNEMP_SRC, skiprows=list(range(11)))

# Start at CPI with 1967 dollars.
df_unemp_orig = df_unemp_orig[df_unemp_orig['Year'] >= LONG_HISTORY_START_YEAR]

# Index by date.
df_unemp_melted = df_unemp_orig.melt(id_vars='Year', var_name='Month', value_name='Value')
df_unemp_melted['Date'] = df_unemp_melted['Year'].astype(str) + '-' + df_unemp_melted['Month']
df_unemp_melted['Date'] = pd.to_datetime(df_unemp_melted['Date'], format='%Y-%b')
df_unemp = df_unemp_melted.set_index('Date')
df_unemp = df_unemp.sort_index()

### Plot

In [ ]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_unemp.index, df_unemp['Value'])

In [ ]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_unemp[df_unemp.index >= MEDIUM_TERM].index, df_unemp[df_unemp.index >= MEDIUM_TERM]['Value']) # Other charts start at 2001.

### Bokeh

In [ ]:
# Plot setup.
unemp_plot = figure(
    height=400,
    width=800,
    title="Unemployment",
    x_axis_label='Date',
    y_axis_label='Rate',
    x_axis_type="datetime",
    tools="",
)

# Date range widget.
unemp_date_range_slider = DateRangeSlider(
    title="Period",
    value=(date(LONG_HISTORY_START_YEAR, 1, 1), date(END_YEAR, 1, 1)), # Hard code for now.
    start=date(LONG_HISTORY_START_YEAR, 1, 1),
    end=date(END_YEAR, 1, 1),
    step=365,
)

# Bind slider to plot for auto updating.
unemp_date_range_slider.js_link('value', unemp_plot.x_range, 'start', attr_selector=0)
unemp_date_range_slider.js_link('value', unemp_plot.x_range, 'end', attr_selector=1)

# Data source.
unemp_source = ColumnDataSource(data={'x': df_unemp.index, 'y': df_unemp['Value'], 'smoothed_y': df_unemp['Value']})

# Line plot.
unemp_line = unemp_plot.line('x', 'y', source=unemp_source, line_width=1, line_alpha=0.2, legend_label='Actual')

# Smoothed line plot.
unemp_smoothed_line = unemp_plot.line('x', 'smoothed_y', source=unemp_source, line_width=1.5, color='red', legend_label='Smooth')

# Resize viewport widget.
unemp_view_size_select = Select(title="Sizing mode", value="fixed", options=list(SizingMode), width=300)
unemp_view_size_select.js_link('value', unemp_plot, 'sizing_mode')

# Resize window widget.
unemp_window_callback = CustomJS(args=dict(source=unemp_source, smoothed_line=unemp_smoothed_line), code="""
    var data = source.data;
    var y = data['y'];
    var smoothed_y = data['smoothed_y'];
    var window_size = cb_obj.value;
    var N = y.length;
    
    // Simple moving average for smoothing
    for (var i = 0; i < N; i++) {
        var sum = 0.0;
        var count = 0;
        for (var j = Math.max(0, i - Math.floor(window_size / 2)); j < Math.min(N, i + Math.ceil(window_size / 2)); j++) {
            sum += y[j];
            count += 1;
        }
        smoothed_y[i] = sum / count;
    }
    
    // Update the plot
    smoothed_line.glyph.change.emit();
""")
unemp_window_size_slider = Slider(start=1, end=100, value=1, step=10, title="Average")
unemp_window_size_slider.js_on_change('value', unemp_window_callback)

# Build layout.
unemp_layout = column(
    unemp_window_size_slider,
    unemp_date_range_slider,
    unemp_view_size_select,
    unemp_plot
)

## Employment Cost Index

Wages and salaries for Private industry workers in All industries and occupations, 12-month percent change

In [ ]:
# Load datasource
df_eci_orig = pd.read_excel(ECI_SRC, skiprows=list(range(15)))

# Insert month from biz quarter.
df_eci_orig['Month'] = df_eci_orig['Period'].apply(monthToQuarter)

# Make date index
df_eci_orig['Date'] = df_eci_orig.apply(lambda row: f"{row['Year']}-{row['Month']}", axis=1)
df_eci_orig['Date'] = pd.to_datetime(df_eci_orig['Date'], format='%Y-%b')
df_eci = df_eci_orig.set_index('Date')
df_eci = df_eci.sort_index()
df_eci.drop(['Standard Error', 'Month', 'Period', 'Year'], axis=1, inplace=True)

### Plot

In [ ]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_eci.index, df_eci['Estimate Value'])

In [ ]:
from scipy.signal import savgol_filter

# window size, polynomial order 
cost_smooth_y = savgol_filter(df_eci['Estimate Value'], 15, 2)

In [ ]:
if (DISABLE_PLOT == False):
    plt.plot(df_eci.index, smooth_y)
    plt.xlabel("Period")
    plt.ylabel("Change")
    plt.show()

## Employment: Num Workers?

In [ ]:
# # Load dataset.
# df_emp_orig = pd.read_excel(EMP_SRC, skiprows=list(range(12)))

# # Start at CPI with 1967 dollars.
# df_emp_orig = df_emp_orig[df_emp_orig['Year'] >= LONG_HISTORY_START_YEAR]

# # Index by date.
# df_emp_melted = df_emp_orig.melt(id_vars='Year', var_name='Month', value_name='Value')
# df_emp_melted['Date'] = df_emp_melted['Year'].astype(str) + '-' + df_emp_melted['Month']
# df_emp_melted['Date'] = pd.to_datetime(df_emp_melted['Date'], format='%Y-%b')
# df_emp = df_emp_melted.set_index('Date')
# df_emp = df_emp.sort_index()
# df_emp.drop(['Year', 'Month'], axis=1, inplace=True)

In [ ]:
# df_emp.head()

In [ ]:
# plt.yscale('linear')
# plt.plot(df_emp.index, df_emp['Value'])

In [ ]:
# plt.yscale('linear')
# plt.plot(df_emp[df_emp.index >= MEDIUM_TERM].index, df_emp[df_emp.index >= MEDIUM_TERM]['Value'])

### Employment: Rate of Change?

In [ ]:
# # Copy df_emp, add rate column.
# df_emp_rate = df_emp.copy()
# df_emp_rate['Rate'] = ((df_emp_rate['Value'] - df_emp_rate['Value'].shift(1)) / df_emp_rate['Value'].shift(1)) * 100
# df_emp_rate.loc['1954-01-01', 'Rate'] = df_emp_rate.loc['1954-02-01', 'Rate']  # Backfill first month

In [ ]:
# plt.yscale('linear')
# plt.ylim((-0.5, 1))
# plt.plot(df_emp_rate.index, df_emp_rate['Rate'])

## Productivity

In [ ]:
# Load dataset.
df_prod_orig = pd.read_excel(PROD_SRC, skiprows=list(range(10)))

# Start at CPI at 1967 dollars.
df_prod_orig = df_prod_orig[df_prod_orig['Year'] >= LONG_HISTORY_START_YEAR]

# Index by date.
df_prod_melted = df_prod_orig.melt(id_vars='Year', var_name='Quarter', value_name='Value')
df_prod_melted['Month'] = df_prod_melted['Quarter'].apply(monthToQuarter)
df_prod_melted['Date'] = df_prod_melted['Year'].astype(str) + '-' + df_prod_melted['Month']
df_prod_melted['Date'] = pd.to_datetime(df_prod_melted['Date'], format='%Y-%b')
df_prod = df_prod_melted.set_index('Date')
df_prod = df_prod.sort_index()
df_prod.drop(['Year', 'Quarter', 'Month'], axis=1, inplace=True)

In [ ]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_prod.index, df_prod['Value'])

In [ ]:
from scipy.signal import savgol_filter

# window size, polynomial order 
prod_smooth_y = savgol_filter(df_prod[df_prod.index >= '2001-01-01']['Value'], 15, 2)

In [ ]:
if (DISABLE_PLOT == False):
    plt.plot(df_prod[df_prod.index >= '2001-01-01'].index, prod_smooth_y)
    plt.xlabel("Period")
    plt.ylabel("Productivity")
    plt.show()

In [ ]:
if (DISABLE_PLOT == False):
    plt.yscale('linear')
    plt.plot(df_prod[df_prod.index >= MEDIUM_TERM].index, df_prod[df_prod.index >= MEDIUM_TERM]['Value'])

## Productivity & Employee Cost

In [ ]:
# Plot setup.
prod_cost_plot = figure(
    height=400,
    width=800,
    x_axis_type="datetime",
    x_axis_label='Period',
    y_axis_label='Productivity and Compensation',
    y_axis_type="linear",
    tools="",
)

# Productivity level.
prod_line = prod_cost_plot.line(df_eci.index, prod_smooth_y, legend_label="Productivity", line_width=2, color="red")

# Employee cost index.
cost_line = prod_cost_plot.line(df_eci.index, cost_smooth_y, legend_label="Compensation", line_width=2)

# Build layout.
prod_cost_layout = column(
    prod_cost_plot
)

## Phillips – Illustration

### Sample dataset

In [ ]:
# Illustration df.
df_phillips_illu = pd.DataFrame({
    'inflation': [15, 8, 7, 4.25, 3, 2.25, 1.625, 1.25, 1, 0.875, 0.625, 0.6, 0.6], 
    'unemployment': [3, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 6.375, 7.5, 8, 14]
})
df_phillips_illu.set_index('unemployment', inplace=True)

# Draws the phillips curve.
def polynomial_func(x, a, b, c):
    return ((a + b) / (x + c * (x**3)))

# Assign x, y data.
x_data = df_phillips_illu.index
y_data = df_phillips_illu['inflation'].values

# Fit a curved line.
x_curve = np.linspace(min(x_data), max(x_data), 100)
y_curve = polynomial_func(x_curve, max(y_data), 100, .6)

### Matplotlib

In [ ]:
if (DISABLE_PLOT == False):

    # Axes.
    plt.xlim(3, 8)
    plt.ylim(0, 8) 
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    
    # Plot curves.
    plt.plot(x_curve, y_curve, color='red', label='Best-fit Curve')
    plt.plot(x_curve, y_curve + 1, color='blue', label='Best-fit Curve')
    plt.plot(x_curve, y_curve + 2, color='green', label='Best-fit Curve')
    
    # Show
    plt.show()

### Bokeh

In [ ]:
# Plot setup.
phillips_illu_plot = figure(
    height=400,
    width=400,
    x_axis_label='Unemployment Rate',
    y_axis_label='Inflation Rate',
    y_axis_type="linear",
    tools="",
)

# Data source.
phillips_source_1 = ColumnDataSource(data={'x': x_curve, 'y': y_curve})
phillips_source_2 = ColumnDataSource(data={'x': x_curve, 'y': y_curve + 1})
phillips_source_3 = ColumnDataSource(data={'x': x_curve, 'y': y_curve + 2})

# Line plot.
phillips_line_1 = phillips_illu_plot.line('x', 'y', source=phillips_source_1, line_width=2, line_alpha=0.5, legend_label='Tight money', hover_alpha=1)
phillips_line_2 = phillips_illu_plot.line('x', 'y', source=phillips_source_2, line_width=2, line_alpha=0.5, legend_label='Neutral money', hover_alpha=1)
phillips_line_3 = phillips_illu_plot.line('x', 'y', source=phillips_source_3, line_width=2, line_alpha=0.5, legend_label='Easy money', hover_alpha=1)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_line_1, phillips_line_2, phillips_line_3], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], line_policy='next')
phillips_illu_plot.add_tools(hover_tool)

# Set x-axis scale.
phillips_illu_plot.x_range.start = 0
phillips_illu_plot.x_range.end = 10

# Set y-axis scale.
phillips_illu_plot.y_range.start = 0
phillips_illu_plot.y_range.end = 10

# X tick intervals
xticker = BasicTicker(desired_num_ticks=3)
phillips_illu_plot.xaxis.ticker = xticker
phillips_illu_plot.xaxis.ticker.num_minor_ticks = 0
phillips_illu_plot.xaxis.axis_line_width = 2
phillips_illu_plot.xaxis.axis_line_color = "#666666"
phillips_illu_plot.xaxis.axis_label_text_color = "#666666"
phillips_illu_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_illu_plot.xaxis.major_label_text_font_size = "9pt"
phillips_illu_plot.xaxis.major_label_text_font_size = "9pt"
phillips_illu_plot.xaxis.major_label_text_font_style = "bold"
phillips_illu_plot.xaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Y tick intervals
yticker = BasicTicker(desired_num_ticks=3)
phillips_illu_plot.yaxis.ticker = yticker
phillips_illu_plot.yaxis.ticker.num_minor_ticks = 0
phillips_illu_plot.yaxis.axis_line_width = 2
phillips_illu_plot.yaxis.axis_line_color = "#666666"
phillips_illu_plot.yaxis.axis_label_text_color = "#666666"
phillips_illu_plot.xaxis.axis_label_text_font_size = "9.5pt"
phillips_illu_plot.yaxis.major_label_text_font_size = "9pt"
phillips_illu_plot.yaxis.major_label_text_font_size = "9pt"
phillips_illu_plot.yaxis.major_label_text_font_style = "bold"
phillips_illu_plot.yaxis.formatter = FuncTickFormatter(code="""
    return `${tick}%`;
""")

# Build layout.
phillips_illu_layout = column(
    phillips_illu_plot
)

## Phillips – Scatters

### Dataset

In [ ]:
# Phillips dataframe.
df_phillips = pd.DataFrame()

# Combine inflation rate and unemployment rate.
df_phillips['inflation_rate'] = df_cpi_rate['Rate'] * 10
df_phillips['unemployment_rate'] = df_unemp['Value']

# Clean up dataframe.
df_phillips.dropna(inplace=True)
df_phillips['Date'] = df_phillips.index
df_phillips.set_index('unemployment_rate', inplace=True)

### Philips - 1954-1970

In [ ]:
# Limit year.
df_phillips_1 = df_phillips[df_phillips['Date'] < '1970-01-01']

#### Matplotlib

In [ ]:
if (DISABLE_PLOT == False):
    plt.scatter(df_phillips_1.index, df_phillips_1['inflation_rate'], s=10)
    plt.xlim(int(min(df_phillips_1.index)), 20)
    plt.ylim(0, 20) 
    plt.yticks(range(0, 11, 10))
    plt.yticks(range(0, 11, 10))
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    plt.show()

#### Bokeh

In [ ]:
# Plot setup.
phillips_plot = figure(
    height=380,
    width=380,
    title="1954-1970",
    x_axis_label='Unemployment',
    y_axis_label='Inflation',
    y_axis_type="linear",
    tools="",
)

# Data source.
phillips_source = ColumnDataSource(data={'x': df_phillips_1.index, 'y': df_phillips_1['inflation_rate']})

# Make it square.
phillips_plot.x_range = Range1d(3.5, 15)
phillips_plot.y_range = Range1d(3.5, 15)

# Line plot.
phillips_circle = phillips_plot.circle('x', 'y', source=phillips_source, size=5, alpha=1)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_circle], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], line_policy='next')
phillips_plot.add_tools(hover_tool)

# Build layout.
phillips_layout_01 = column(
    phillips_plot
)

# Hide legend.
phillips_plot.legend.visible=False

### Phillips - 1970-1986

In [ ]:
# Limit year.
df_phillips_2 = df_phillips[(df_phillips['Date'] >= '1970-01-01') & (df_phillips['Date'] < '1986-01-01')]

#### Matplotlib

In [ ]:
if (DISABLE_PLOT == False):
    plt.scatter(df_phillips_2.index, df_phillips_2['inflation_rate'], s=10)
    plt.xlim(int(min(df_phillips_2.index)), 20)
    plt.ylim(0, 20) 
    plt.yticks(range(0, 11, 10))
    plt.yticks(range(0, 11, 10))
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    plt.show()

#### Bokeh

In [ ]:
# Plot setup.
phillips_plot = figure(
    height=380,
    width=380,
    title="1970-1986",
    x_axis_label='Unemployment',
    y_axis_label='Inflation',
    y_axis_type="linear",
    tools="",
)

# Data source.
phillips_source = ColumnDataSource(data={'x': df_phillips_2.index, 'y': df_phillips_2['inflation_rate']})

# Make it square.
phillips_plot.x_range = Range1d(3.5, 15)
phillips_plot.y_range = Range1d(3.5, 15)

# Line plot.
phillips_circle = phillips_plot.circle('x', 'y', source=phillips_source, size=5)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_circle], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], line_policy='next')
phillips_plot.add_tools(hover_tool)

# Build layout.
phillips_layout_02 = column(
    phillips_plot
)

# Hide legend.
phillips_plot.legend.visible=False

### Phillips - 1986-2000

In [ ]:
# Limit year.
df_phillips_3 = df_phillips[(df_phillips['Date'] >= '1986-01-01') & (df_phillips['Date'] < '2000-01-01')]

#### Matplotlib

In [ ]:
if (DISABLE_PLOT == False):
    plt.scatter(df_phillips_3.index, df_phillips_3['inflation_rate'], s=10)
    plt.xlim(int(min(df_phillips_3.index)), 20)
    plt.ylim(0, 20) 
    plt.yticks(range(0, 11, 10))
    plt.yticks(range(0, 11, 10))
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    plt.show()

#### Bokeh

In [ ]:
# Plot setup.
phillips_plot = figure(
    height=380,
    width=380,
    title="1986-2000",
    x_axis_label='Unemployment',
    y_axis_label='Inflation',
    y_axis_type="linear",
    tools="",
)

# Data source.
phillips_source = ColumnDataSource(data={'x': df_phillips_3.index, 'y': df_phillips_3['inflation_rate']})

# Make it square.
phillips_plot.x_range = Range1d(3.5, 15)
phillips_plot.y_range = Range1d(3.5, 15)

# Line plot.
phillips_circle = phillips_plot.circle('x', 'y', source=phillips_source, size=5)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_circle], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], line_policy='next')
phillips_plot.add_tools(hover_tool)

# Build layout.
phillips_layout_03 = column(
    phillips_plot
)

# Hide legend.
phillips_plot.legend.visible=False

### Phillips - 1996-2013

In [ ]:
# Limit year.
df_phillips_4 = df_phillips[(df_phillips['Date'] >= '1996-01-01') & (df_phillips['Date'] < '2013-01-01')]

#### Matplotlib

In [ ]:
if (DISABLE_PLOT == False):
    plt.scatter(df_phillips.index, df_phillips['inflation_rate'], s=10)
    plt.xlim(int(min(df_phillips.index)), 20)
    plt.ylim(0, 20) 
    plt.yticks(range(0, 11, 10))
    plt.yticks(range(0, 11, 10))
    plt.xlabel('Inflation Rate (%)', fontsize=10, color='black')
    plt.ylabel('Unemployment Rate (%)', fontsize=10, color='black')
    plt.show()

#### Bokeh

In [ ]:
# Plot setup.
phillips_plot = figure(
    height=380,
    width=380,
    title="1996-2013",
    x_axis_label='Unemployment',
    y_axis_label='Inflation',
    y_axis_type="linear",
    tools="",
)

# Data source.
phillips_source = ColumnDataSource(data={'x': df_phillips_4.index, 'y': df_phillips_4['inflation_rate']})

# Make it square.
phillips_plot.x_range = Range1d(3.5, 15)
phillips_plot.y_range = Range1d(3.5, 15)

# Line plot.
phillips_circle = phillips_plot.circle('x', 'y', source=phillips_source, size=5)

# Increase alpha on hover.
hover_tool = HoverTool(renderers=[phillips_circle], tooltips=[("Unemployment Rate", "@x{0.0}%"), ("Inflation Rate", "@y{0.0}%")], line_policy='next')
phillips_plot.add_tools(hover_tool)

# Build layout.
phillips_layout_04 = column(
    phillips_plot
)

# Hide legend.
phillips_plot.legend.visible=False

# Layout

### Text content

In [ ]:
intro_text = Div(
    text="""
        <h1>Phillips Curve?</h1>
        <h2>This is the subtitle of the article</h2>
        <p>Cum sociis natoque penatibus et magnis dis parturient montes, 
        nascetur ridiculus mus. Integer posuere erat a ante venenatis 
        dapibus posuere velit aliquet. Lorem ipsum dolor sit amet, 
        consectetur adipiscing elit. Aenean lacinia bibendum nulla sed 
        consectetur. Cras justo odio, dapibus ac facilisis in, egestas eget quam. 
        Vivamus sagittis lacus vel augue laoreet rutrum faucibus dolor auctor. 
        Nullam id dolor id nibh ultricies vehicula ut id elit.</p>
        """,
    # width=740,
    stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
)

In [ ]:
body_section_0 = Div(
    text="""
        <h2>Nullam id dolor id nibh.</h2>
        <p>Lorem ipsum dolor sit amet, 
        consectetur adipiscing elit. Aenean lacinia bibendum nulla sed 
        consectetur. Cras justo odio, dapibus ac facilisis in, egestas eget quam. 
        Vivamus sagittis lacus vel augue laoreet rutrum faucibus dolor auctor. 
        Nullam id dolor id nibh ultricies vehicula ut id elit.</p>
        <p>Vestibulum id ligula porta felis euismod semper. Donec sed odio dui. 
        Curabitur blandit tempus porttitor. Donec ullamcorper nulla non metus auctor 
        fringilla. Nullam quis risus eget urna mollis ornare vel eu leo.</p>
        <h2>This is a real heading</h2>
        <p>Nullam id dolor id nibh ultricies vehicula ut id elit. Vestibulum id ligula 
        porta felis euismod semper. Donec sed odio dui. Donec ullamcorper nulla non 
        metus auctor fringilla. Fusce dapibus, tellus ac cursus commodo, tortor mauris 
        condimentum nibh, ut fermentum massa justo sit amet risus. Duis mollis, est 
        non commodo luctus, nisi erat porttitor ligula, eget lacinia odio sem nec 
        elit.</p>
        """,
    # width=680,
    stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
)

In [ ]:
body_section_1 = Div(
    text="""
        <h2>When headings were headings.</h2>
        <p>Lorem ipsum dolor sit amet, 
        consectetur adipiscing elit. Aenean lacinia bibendum nulla sed 
        consectetur. Cras justo odio, dapibus ac facilisis in, egestas eget quam. 
        Vivamus sagittis lacus vel augue laoreet rutrum faucibus dolor auctor. 
        Nullam id dolor id nibh ultricies vehicula ut id elit.</p>
        <p>Vestibulum id ligula porta felis euismod semper. Donec sed odio dui. 
        Curabitur blandit tempus porttitor. Donec ullamcorper nulla non metus auctor 
        fringilla. Nullam quis risus eget urna mollis ornare vel eu leo.</p>
        <h2>This is a real heading</h2>
        <p>Nullam id dolor id nibh ultricies vehicula ut id elit. Vestibulum id ligula 
        porta felis euismod semper. Donec sed odio dui. Donec ullamcorper nulla non 
        metus auctor fringilla. Fusce dapibus, tellus ac cursus commodo, tortor mauris 
        condimentum nibh, ut fermentum massa justo sit amet risus. Duis mollis, est 
        non commodo luctus, nisi erat porttitor ligula, eget lacinia odio sem nec 
        elit.</p>
        <h2>But who needs headings?</h2>
        <p>Maecenas sed diam eget risus varius blandit sit amet non magna. Etiam 
        porta sem malesuada magna mollis euismod. Nulla vitae elit libero, a pharetra 
        augue. Praesent commodo cursus magna, vel scelerisque nisl consectetur et. 
        Praesent commodo cursus magna, vel scelerisque nisl consectetur et. 
        Nulla vitae elit libero, a pharetra augue.</p>
        """,
    width=680,
    stylesheets=[InlineStyleSheet(css="p { font-size: 16px; margin: 1em 0; }")],
)

In [ ]:
phillips_text_1 = Div(
    text="""
        <h2>Phillips in 1954-1986</h2>
        <p>Cum sociis natoque penatibus et magnis dis parturient montes, 
        nascetur ridiculus mus. Integer posuere erat a ante venenatis 
        dapibus posuere velit aliquet. Lorem ipsum dolor sit amet, 
        consectetur adipiscing elit.</p>
        <p>Aenean lacinia bibendum nulla sed 
        consectetur. Cras justo odio, dapibus ac facilisis in, egestas eget quam. 
        Vivamus sagittis lacus vel augue laoreet rutrum faucibus dolor auctor. 
        Nullam id dolor id nibh ultricies vehicula ut id elit.</p>
        """,
    width=400,
    stylesheets=[InlineStyleSheet(css="p { font-size: 14px; margin: 1em 0; }")],
)

In [ ]:
phillips_text_2 = Div(
    text="""
        <h2>Phillips in 1986-2013</h2>
        <p>Sed posuere consectetur est at lobortis. Integer posuere erat a ante 
        venenatis dapibus posuere velit aliquet. Praesent commodo cursus 
        magna, vel scelerisque nisl consectetur et. Donec ullamcorper nulla non 
        metus auctor fringilla. Aenean eu leo quam. Pellentesque ornare sem 
        lacinia quam venenatis vestibulum. Donec id elit non mi porta gravida at 
        eget metus.</p>
        <h3>A possible repeat of the prior period?</h3>
        <p>Aenean lacinia bibendum nulla sed 
        consectetur. Cras justo odio, dapibus ac facilisis in, egestas eget quam. 
        Vivamus sagittis lacus vel augue laoreet rutrum faucibus dolor auctor. 
        Nullam id dolor id nibh ultricies vehicula ut id elit.</p>
        """,
    width=400,
    stylesheets=[InlineStyleSheet(css="p { font-size: 14px; margin: 1em 0; }")],
)

## Page

In [ ]:
cpi_unempl_layout = column(
    row(
        column(
            intro_text,
            body_section_0,
            width=820,
            styles=Styles(
                padding='30px 30px 30px 60px',   
            )
        ),
        column(
            Div(
                text="""
                <img src="phillip_portrait.png" />
                <p>William Phillips, originator of the "Phillips Curve." <em>Credit: <a href="https://en.wikipedia.org/wiki/Bill_Phillips_(economist)" target="_blank">Wikipedia</a></em></p>
                """,
                stylesheets=[InlineStyleSheet(css="img { max-width: 100%; height: auto; padding: 15px; border: 1px solid #ccc; margin-top: 60px; }")],
            ),
            width=500,
            styles=Styles(
                padding='0 60px 0 30px',
            ),
        ),
    ),
    row(
        column(
            prod_cost_layout,
        ),
    ),
    row(
        column(
            cpi_rate_plot,
        ),
        column(
            cpi_rate_window_size_slider,
            cpi_rate_date_range_slider,
            styles=Styles(
                padding='30px',
                margin_top='auto',
                margin_bottom='auto',
            ),
        ),
        styles=Styles(
            padding='30px',
        ),
    ),
    row(
        column(
            unemp_plot,
        ),
        column(
            unemp_window_size_slider,
            unemp_date_range_slider,
            styles=Styles(
                padding='30px',
                margin_top='auto',
                margin_bottom='auto',
            ),
        ),
        styles=Styles(
            padding='30px',
        ),
    ),
    row(
        column(
            body_section_1,
            styles=Styles(
                margin='30px 60px 30px 75px',
            )
        ),
        column(
            Div(
                text="""
                    <h2>Does monetary policy hit the curve?</h2>
                    <p>Many economists claim expansionary "easy" monetary policy tends to
                    shift the Phillips Curve up to compensate for the new
                    liquidity, leaving unemployment mostly unchanged.</p>
                    """,
                width=400,
                styles=Styles(
                    padding='0 0 0 30px',
                ),
                stylesheets=[InlineStyleSheet(css="p { font-size: 14px; margin: 1em 0; }")],
            ),
            phillips_illu_layout,
            styles=Styles(
                margin='auto 0',
            ),
        )
    ),
    row(
        column(
            phillips_layout_01,
        ),
        column(
            phillips_layout_02,
        ),
        column(
            phillips_text_1,
            styles=Styles(
                padding='30px 30px 30px 60px',
            ),
        ),
        styles=Styles(
            padding='30px',
        ),
    ),
    row(
        column(
            phillips_layout_03,
        ),
        column(
            phillips_layout_04,
        ),
        column(
            phillips_text_2,
            styles=Styles(
                padding='30px 30px 30px 60px',
            ),
        ),
        styles=Styles(
            padding='0 30px 30px',
        ),
    ),
    styles=Styles(
        border_width='1px',
        border_style='solid',
        border_color='#cccccc',
        margin='60px',
    ),
)

# Stretch both.
cpi_unempl_gridplot = gridplot([[cpi_unempl_layout]])

# Save layout.
output_file("index.html")
save(cpi_unempl_gridplot)